In [8]:
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.models import Model
import tf2onnx
import onnxruntime as ort
import numpy as np
import os
from tensorflow.keras.datasets import cifar10

In [9]:
BATCH_SIZE = 32
IMG_SIZE = (224, 224)
DATA_DIR = "skin_cancer" 
TRAIN_DIR = os.path.join(DATA_DIR, "train")
TEST_DIR = os.path.join(DATA_DIR, "test")

In [11]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DIR,
    shuffle=True,
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    TEST_DIR,
    shuffle=False, 
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE
)

Found 2967 files belonging to 3 classes.
Found 330 files belonging to 3 classes.


In [12]:
class_names = train_ds.class_names
print(f"Classes found: {class_names}")

Classes found: ['Benign keratosis-like lesions', 'Melanocytic nevi', 'Melanoma']


In [ ]:
def build_transfer_model(num_classes=3):
    inputs = Input(shape=(224, 224, 3))

    
    x = layers.RandomFlip("horizontal")(inputs)
    x = layers.RandomRotation(0.1)(x)

    x = preprocess_input(x)

    base_model = ResNet50(include_top=False, weights='imagenet')
    base_model.trainable = False # Freeze the pretrained weights
    
    x = base_model(x, training=False)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.2)(x)  
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs, outputs)
    return model